In [1]:
import numpy as np
import pygimli as pg
import pygimli.meshtools as mt
from pygimli.physics import ert

In [96]:
eldist = 0.023
eldia = 0.0075
size2 = 0.1
plc = mt.createWorld(start=[-size2, -size2, -size2], end=[size2+3*eldist, size2, 0])
for i in range(4):
    pad = mt.createFacet(mt.createCircle(pos=[i*eldist, 0, 0], radius=eldia/2, isHole=True)) 
    plc.copyBoundary(pad.boundary(0))
    plc.createBoundary(plc.nodes([plc.createNode(n.pos()).id() for n in pad.nodes() ]), marker=-10000-i) # -10000-1)

mesh = mt.createMesh(plc, quality=1.3)
mesh.node(0).setMarker(-999)
mesh.node(1).setMarker(-1000)
print(np.unique(mesh.boundaryMarkers()))
pg.show(mesh, showMesh=True, markers=True, backend="trame")

25/10/23 - 15:35:00 - pyGIMLi - WARNING - Given data fits neither cell count nor node count:
25/10/23 - 15:35:00 - pyGIMLi - WARNING - 5741 vs. 0 vs. 361


[-10003 -10002 -10001 -10000     -2     -1      0]


Widget(value="<iframe src='http://localhost:59121/index.html?ui=P_0x217678df0d0_41&reconnect=auto' style='widt…

(<pyvista.plotting.plotting.Plotter at 0x217678df0d0>, None)

In [97]:
shm = pg.DataContainerERT()
for i in range(4):
    shm.createSensor([i*eldist, 0, 0])

shm.addFourPointData(0, 3, 1, 2)
print(shm)
shm["k"] = 1.

Data: Sensors: 4 data: 1, nonzero entries: ['a', 'b', 'm', 'n', 'valid']


In [98]:
data = ert.simulate(mesh, shm, res=1.0, verbose=True)#, calcOnly=True, returnFields=True)  # res=[[1, 1.0]]

RuntimeError:  The potential can not determined directly from ElectrodeShapeDomain use the matrix extension from complete electrode model instead

In [99]:
fop = ert.ERTModelling()
fop.setData(shm)
fop.setMesh(mesh)
model = np.ones(mesh.cellCount())
fop.createJacobian(model)
sens = fop.jacobian()[0]

25/10/23 - 15:38:04 - pyGIMLi - INFO - Found 1 regions.
25/10/23 - 15:38:04 - pyGIMLi - INFO - Creating forward mesh from region infos.
25/10/23 - 15:38:05 - pyGIMLi - INFO - Creating refined mesh (H2) to solve forward task.
25/10/23 - 15:38:05 - pyGIMLi - INFO - Mesh for forward task: Mesh: Nodes: 4321 Cells: 21528 Boundaries: 22964


RuntimeError:  The potential can not determined directly from ElectrodeShapeDomain use the matrix extension from complete electrode model instead